/* Welcome to the SQL mini project. You will carry out this project partly in
the PHPMyAdmin interface, and partly in Jupyter via a Python connection.

This is Tier 1 of the case study, which means that there'll be more guidance for you about how to 
setup your local SQLite connection in PART 2 of the case study. 

The questions in the case study are exactly the same as with Tier 2. 

PART 1: PHPMyAdmin
You will complete questions 1-9 below in the PHPMyAdmin interface. 
Log in by pasting the following URL into your browser, and
using the following Username and Password:

URL: https://sql.springboard.com/
Username: student
Password: learn_sql@springboard

The data you need is in the "country_club" database. This database
contains 3 tables:
    i) the "Bookings" table,
    ii) the "Facilities" table, and
    iii) the "Members" table.

In this case study, you'll be asked a series of questions. You can
solve them using the platform, but for the final deliverable,
paste the code for each solution into this script, and upload it
to your GitHub.

Before starting with the questions, feel free to take your time,
exploring the data, and getting acquainted with the 3 tables. */

In [ ]:
QUESTIONS:

Q1: Some of the facilities charge a fee to members, but some do not.
Write a SQL query to produce a list of the names of the facilities that do. 

Answer 1:

SELECT name 
FROM Facilities 
WHERE membercost >0;

Q2: How many facilities do not charge a fee to members? 

Amswer 2: 

SELECT name
FROM Facilities   
WHERE membercost = 0;

Q3: Write an SQL query to show a list of facilities that charge a fee to members, where the fee is less than 20% of the facility's monthly maintenance cost. Return the facid, facility name, member cost, and monthly maintenance of the facilities in question.

Answer 3:

SELECT facid, name, membercost, monthlymaintenance
FROM Facilities
WHERE membercost >0
AND membercost < (monthlymaintenance * 0.2)
LIMIT 0 , 30;

Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5. Try writing the query without using the OR operator.

Answer 4:

SELECT *
FROM Facilities
WHERE facid >0
AND facid <6
AND facid NOT
IN (2, 3, 4)
LIMIT 0 , 30;

Q5: Produce a list of facilities, with each labelled as 'cheap' or 'expensive', depending on if their monthly maintenance cost is more than $100. Return the name and monthly maintenance of the facilities in question. 

Answer 5:

SELECT name, `monthlymaintenance` ,
CASE WHEN `monthlymaintenance` >100 THEN 'expensive'
WHEN `monthlymaintenance` <100 THEN 'cheap' ELSE ' '
END AS Label
FROM Facilities
LIMIT 0 , 30

Q6: You'd like to get the first and last name of the last member(s)
who signed up. Try not to use the LIMIT clause for your solution. 

Answer 6:

SELECT `surname` , `firstname`
FROM Members
WHERE `joindate` = (SELECT MAX( `joindate`) FROM Members)

Q7: Produce a list of all members who have used a tennis court.
Include in your output the name of the court, and the name of the member
formatted as a single column. Ensure no duplicate data, and order by
the member name.

Answer 7:

SELECT DISTINCT CONCAT( firstname, ' ', surname ) AS MemberName, name AS TennisFacility
FROM Bookings
INNER JOIN Facilities ON Bookings.facid = Facilities.facid
INNER JOIN Members ON Bookings.memid = Members.memid
WHERE name = 'Tennis Court 1'
OR name = 'Tennis Court 2'
ORDER BY MemberName;

Q8: Produce a list of bookings on the day of 2012-09-14 which will cost the member (or guest) more than $30. Remember that guests have different costs to members (the listed costs are per half-hour 'slot'), and the guest user's ID is always 0. Include in your output the name of the facility, the name of the member formatted as a single column, and the cost. Order by descending cost, and do not use any subqueries. 

Answer 8:

SELECT f.name AS FacilityName, CONCAT(m.firstname,' ',m.surname) AS CustomerName, CASE WHEN b.memid=0 THEN (f.guestcost * b.slots) ELSE (f.membercost * b.slots) END AS cost
FROM Bookings AS b
LEFT JOIN Members AS m ON m.memid = b.memid
LEFT JOIN Facilities AS f ON f.facid = b.facid
WHERE b.starttime LIKE '2012-09-14%' 
AND CASE WHEN b.memid =0 THEN (f.guestcost *b.slots) > 30 ELSE (f.membercost * b.slots) > 30 END
ORDER BY CASE WHEN b.memid =0 THEN (f.guestcost *b.slots) ELSE (f.membercost * b.slots) END DESC

Q9: This time, produce the same result as in Q8, but using a subquery. 

Answer 9:

SELECT *
FROM (SELECT f.name, SUM(CASE WHEN b.memid !=0 THEN (f.membercost * b.slots) ELSE (f.guestcost * b.slots) END) AS total_revenue
FROM Facilities AS f
LEFT JOIN Bookings AS b ON f.facid = b.facid
GROUP BY f.name
) AS subquery
WHERE total_revenue <1000
ORDER BY total_revenue


PART 2: SQLite

We now want you to jump over to a local instance of the database on your machine. Copy and paste the LocalSQLConnection.py script into an empty Jupyter notebook, and run it. 

Make sure that the SQLFiles folder containing thes files is in your working directory, and that you haven't changed the name of the .db file from 'sqlite\db\pythonsqlite'.

You should see the output from the initial query 'SELECT * FROM FACILITIES'.
Complete the remaining tasks in the Jupyter interface. If you struggle, feel free to go back to the PHPMyAdmin interface as and when you need to. You'll need to paste your query into value of the 'query1' variable and run the code block again to get an output.
 

Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members! 

Answer 10:

SELECT *
FROM (SELECT f.name, SUM(CASE WHEN b.memid != 0 THEN (f.membercost *b.slots) ELSE (f.guestcost * b.slots) END) AS total_revenue
FROM Facilities As f
LEFT JOIN Bookings AS b ON f.facid = b.facid
GROUP BY f.name
) AS subquery
WHERE total_revenue < 1000
ORDER BY total_revenue;

Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order 

Answer 11:

SELECT CONCAT(m.surname,',',m.firstname) AS MemberName, CONCAT(sub.surname,',',sub.firstname) AS RecommenderName
From Members AS m
LEFT JOIN Members AS sub ON m.recommendedby = sub.memid
ORDER BY MemberName;

Q12: Find the facilities with their usage by member, but not guests 

Answer 12:

SELECT f.name, SUM(CASE WHEN b.memid = 0 THEN 0 ELSE 1 END) AS Facility_Usage
FROM Bookings AS b
LEFT JOIN Facilities AS f ON f.facid = b.facid
GROUP BY f.facid;

Q13: Find the facilities usage by month, but not guests 

Answer 13:

SELECT f.name AS Facility_Name, MONTH(b.starttime) AS month, SUM(CASE WHEN b.memid =0 THEN 0 ELSE 1 END) Monthly_Facility_Use
FROM Bookings AS b
LEFT JOIN Facilities AS f ON f.facid = b.facid
GROUP BY Facility_Name
ORDER BY month;
